<a href="https://colab.research.google.com/github/vgctelles/spotify-model2/blob/main/spotify_data_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth


In [55]:
CLIENT_ID = 'dcdbc364b650420ba00c3fbcca957a4f'
CLIENT_SECRET = '381f0fa966784f4e92b37134958856d4'
SPOTIPY_REDIRECT_URI ='https://www.kaggle.com/vgctelles/spotify-data-scrap/'
#SPOTIPY_REDIRECT_URI = 'http://localhost/'
#REDIRECT_URL = 'http://127.0.0.1:9090'
SCOPE = "user-library-read"

In [34]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify( client_credentials_manager=client_credentials_manager)

In [6]:
#test 1000 data scrap

artist_name = []
track_name = []
track_popularity = []
track_explicity = []
artist_id = []
track_id = []
album_name = []
album_id = []
        #range(start, stop, step)
for i in range(0,1000,50):
    track_results = sp.search(q='year:2021', type='track', limit=50, market = 'BR', offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        album_name.append(t['album']['name'])
        album_id.append(t['album']['id'])
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

In [8]:
track_df = pd.DataFrame({'album_name' : album_name,'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'track_popularity' : track_popularity, 'explicit' : track_explicity, 'artist_id' : artist_id})
print(track_df.shape)

(1000, 7)


In [9]:
track_df.head()

,album_name,artist_name,track_name,track_id,track_popularity,explicit,artist_id
0,Molhando o Volante,Jorge & Mateus,Molhando o Volante,3ITkrUvAA6gzxer2UIom3P,82,False,1elUiq4X7pxej6FRlrEzjM
1,Merry Christmas and Happy New Year 2021 (Radio...,Dj Biffen,Merry Christmas and Happy New Year 2021 - Radi...,7h7yPyss4MIBVPXVeF83vg,23,True,7K54lZmkO9dwqe22N27ftb
2,Malvadão 3,Xamã,Malvadão 3,0NBgyml7T0IvUNbXIkcSpH,82,False,5YwzDz4RJfTiMHS4tdR5Lf
3,Lofi Fruits Music 2021,Lofi Fruits Music,7 Years,2JjwQsLvMknwKhMRbdC7Mb,52,False,1dABGukgZ8XKKOdd2rVSHM
4,Vai Lá Em Casa Hoje,George Henrique & Rodrigo,Vai Lá Em Casa Hoje,2wSCalnGOnXdAJC0Fn8OUY,81,False,3FlCKtwOEYQQYyGVEbxG2Z


In [10]:
track_df.count()

album_name          1000
artist_name         1000
track_name          1000
track_id            1000
track_popularity    1000
explicit            1000
artist_id           1000
dtype: int64

In [11]:
artist_popularity = []
artist_genres = []
artist_followers = []
for a_id in track_df.artist_id:
  artist = sp.artist(a_id)
  artist_popularity.append(artist['popularity'])
  artist_genres.append(artist['genres'])
  artist_followers.append(artist['followers']['total'])

In [12]:
track_df = track_df.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)
track_df.head()

,album_name,artist_name,track_name,track_id,track_popularity,explicit,artist_id,artist_popularity,artist_genres,artist_followers
0,Molhando o Volante,Jorge & Mateus,Molhando o Volante,3ITkrUvAA6gzxer2UIom3P,82,False,1elUiq4X7pxej6FRlrEzjM,79,"[arrocha, sertanejo, sertanejo universitario]",16728862
1,Merry Christmas and Happy New Year 2021 (Radio...,Dj Biffen,Merry Christmas and Happy New Year 2021 - Radi...,7h7yPyss4MIBVPXVeF83vg,23,True,7K54lZmkO9dwqe22N27ftb,17,[],26
2,Malvadão 3,Xamã,Malvadão 3,0NBgyml7T0IvUNbXIkcSpH,82,False,5YwzDz4RJfTiMHS4tdR5Lf,73,"[brazilian hip hop, funk carioca, trap brasile...",1690059
3,Lofi Fruits Music 2021,Lofi Fruits Music,7 Years,2JjwQsLvMknwKhMRbdC7Mb,52,False,1dABGukgZ8XKKOdd2rVSHM,74,"[lo-fi cover, lo-fi product]",534824
4,Vai Lá Em Casa Hoje,George Henrique & Rodrigo,Vai Lá Em Casa Hoje,2wSCalnGOnXdAJC0Fn8OUY,81,False,3FlCKtwOEYQQYyGVEbxG2Z,67,"[arrocha, sertanejo, sertanejo pop, sertanejo ...",1341031


In [13]:
track_features = []
for t_id in track_df['track_id']:
  af = sp.audio_features(t_id)
  track_features.append(af)
tf_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
for item in track_features:
  for feat in item:
    tf_df = tf_df.append(feat, ignore_index=True)

In [ ]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlists = sp.user_playlists('spotify')

In [57]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = CLIENT_ID, client_secret = CLIENT_SECRET, redirect_uri = SPOTIPY_REDIRECT_URI, scope=scope))

results = sp.current_user_playlists(limit=2, offset = 1)

#auth_manager=SpotifyOAuth()
#    client_id = CLIENT_ID, client_secret = CLIENT_SECRET, scope = SCOPE, redirect_uri = REDIRECT_URL)
#sp.set_auth(auth_manager)

Enter the URL you were redirected to:  


SpotifyOauthError: error: invalid_request, error_description: code must be supplied

In [ ]:
teste = sp.current_user_recently_played()

In [ ]:
playlists = sp.current_user_playlists(limit=2, offset = 1)

In [ ]:
playlist